# P5 - Convolutioneel neuraal netwerk 

## Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

## Load the CIFAR-10 dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

## Prepare data

In [ ]:
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

## Build Convolutional Neural Network
### Input shape
De `input_shape` is (32, 32, 3) omdat dit staat voor de grootte van de input(32x32 foto's) en de 3 kleurkanalen (RGB).
### Eerste convolutionele laag
- 32 filters: Deze laag leert 32 kenmerken hiervan.
- 3x3 kernel size: 3x3 wordt vaak gebruikt bij foto herkenning. Het is een goede balans tussen details kunnen herkennen, maar ook niet teveel rekenkracht nodig hebben.
- ReLU: ReLU (Rectified Linear Unit): Deze rectifier maakt alle negatieve outputs 0. Dit [omzeilt sommige problemen die kunnen opkomen en is best snel](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)#Advantages).
### Max pooling laag
- 2x2: Dit convert 4 inputs naar 1. Dit maakt het netwerk sneller, maar met verlies van precisie.
### Tweede en derde convolutionele laag
- 64/128 filters: De filters worden steeds groter, omdat er op hogere niveaus meer complexe kenmerken vastgelegd willen worden.
### Tweede en derde pooling laag
- Er wordt steeds weer pooling toegepast om het model sneller te maken.
### Flatten
- Dit maakt van de outputs een 1D vector, wat makkelijker is om mee te werken.
### Dense
- 512 neurons: Dense maakt een neurale netwerk, met in dit geval 512 neurons. 512 wordt gekozen omdat er nog best veel informatie is, maar meer dan dat is ook niet nodig.
### Dropout
- Dit wordt gebruikt om 0.5 (50%) van de neurons naar 0 te zetten. Dit voorkomt overfitting, omdat als dit niet gedaan wordt het model afhankelijk kan worden van specifieke neurons.
### Dense 2 (output laag)
- 10 neurons: Er zijn 10 klassen in de CIFAR-10 dataset, daarom ook 10 neurons in de output laag.
### Compilatie
- `categorical_crossentropy`: Dit wordt vaak gebruikt voor multi-class classificatie, zoals CIFAR-10, waar de input behoort tot 1 van meerdere klasses.
- Adam: Adam (Adaptive Moment Estimation) is een veel gebruikte optimizer die de learning rate aanpast gebaseerd op de gradients.

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train model
### Epochs
- 10: Er is gekozen voor 10 epochs omdat meer dan 10 last heeft van overfitting. Zo is bij 50 epochs de accuracy 72%, terwijl dit bij 10 epochs 73% is.
### Batch size
- Kleine batch size (64): Er is gekozen voor een kleine batch size omdat dit problemen zoals overfitting omzeilt. Het nadeel is dat grotere batch sizes veel stabieler gradients krijgen.

In [ ]:
# accuracy 50 epochts = ~0.72, 10 epochs = ~0.73
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

## Conclusie
### Accuracy
Aan deze plots is te zien dat er geen overfitting is. Met bijvoorbeeld `epochs=50` is gelijk te zien dat er wel overfitting is, omdat bij de accuracy dan de blauwe en oranje curve niet dichtbij elkaar zijn. Hierbij is de blauwe curve rond de 90% en de oranje curve rond de 70%.

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()